#Install BERTweet and needed libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install emoji

     |████████████████████████████████| 1.9MB 5.4MB/s 
     |████████████████████████████████| 890kB 42.1MB/s 
     |████████████████████████████████| 3.2MB 33.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=e8f1bc1a54368bccc22cb917faa714686de1127d3415b8172ce615832898c06c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 6.0MB/s 
     |████████████████████████████████| 133kB 5.5MB/s 


In [ ]:
import torch
import os
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
import re
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

#Data Processing

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/UPENN/Task6/train_valid.tsv", sep='\t')
df_train.columns = ["tweet_id", "tweet", "label"]
df_train.head()

,tweet_id,tweet,label
0,12399,Medical experts advise that symptoms of the no...,Lit-News_mentions
1,20056,@drdavidsamadi Hubby/I:same symptoms n Novembe...,Nonpersonal_reports
2,10175,1/x In the April 11 BC briefing Dr. Bonnie Hen...,Lit-News_mentions
3,12179,Major study PHOSP-COVID investigates health im...,Lit-News_mentions
4,30823,I’ve had 4 waves of symptoms over 7 weeks. No ...,Self_reports


In [ ]:
# df_valid = pd.read_csv("/content/drive/MyDrive/UPENN/Task6/valid.tsv", sep='\t')
# df_valid.columns = ["tweet_id", "tweet", "label"]
# df_valid.head()

,tweet_id,tweet,label
0,13361,Loss of taste and smell is best indicator of C...,Lit-News_mentions
1,20228,Me and my girl swear we have already had COVID...,Nonpersonal_reports
2,20120,It’s great to have some guidelines around fati...,Nonpersonal_reports
3,23604,My partner &amp; I had #coronavirus in late Fe...,Nonpersonal_reports
4,31412,Covid week 13 update. Week 11 kidney pain on t...,Self_reports


In [ ]:
df_train.label.value_counts()

Lit-News_mentions      4523
Nonpersonal_reports    3622
Self_reports           1421
Name: label, dtype: int64

In [ ]:
df_train['label'].replace({"Lit-News_mentions":0, "Nonpersonal_reports":1, "Self_reports":2}, inplace=True)
# df_valid['label'].replace({"Lit-News_mentions":0, "Nonpersonal_reports":1, "Self_reports":2}, inplace=True)
df_train.head()

,tweet_id,tweet,label
0,12399,Medical experts advise that symptoms of the no...,0
1,20056,@drdavidsamadi Hubby/I:same symptoms n Novembe...,1
2,10175,1/x In the April 11 BC briefing Dr. Bonnie Hen...,0
3,12179,Major study PHOSP-COVID investigates health im...,0
4,30823,I’ve had 4 waves of symptoms over 7 weeks. No ...,2


In [ ]:
# X_train,X_val, Y_train, Y_val= train_test_split(df['tweet'], df['label'], test_size = 0.2, random_state = 42)
# X_train = X_train.reset_index(drop=True)
# X_val = X_val.reset_index(drop=True)
# Y_train = Y_train.reset_index(drop=True)
# Y_val = Y_val.reset_index(drop=True)

X_train = np.array(df_train['tweet'])
# X_val = np.array(df_valid['tweet'])
Y_train = np.array(df_train['label'])
# Y_val = np.array(df_valid['label'])

In [ ]:
X_train = X_train.tolist()
# X_val = X_val.tolist()
Y_train.tolist()
# Y_val.tolist()

[0,
 1,
 0,
 0,
 2,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 1,
 2,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 2,
 2,
 1,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 2,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 2,
 1,
 0,
 0,
 1,
 0,


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
bertweet = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-covid19-base-uncased", num_labels=3)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Some weights of the model checkpoint at vinai/bertweet-covid19-base-uncased were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-covid19-base-unc

In [ ]:
train_encodings = tokenizer(X_train,truncation=True, padding=True)
# val_encodings = tokenizer(X_val,truncation=True, padding=True)

In [ ]:
class Task6Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Task6Dataset(train_encodings, Y_train)
# val_dataset = Task6Dataset(val_encodings, Y_val)

In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=12,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    logging_steps=10,
)

trainer = Trainer(
    model=bertweet,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    compute_metrics = compute_metrics
)

trainer.train()

Step,Training Loss
10,1.129700
20,1.112500
30,1.079300
40,1.041800
50,1.015900
60,0.948000
70,0.855300
80,0.719600
90,0.637000
100,0.529300


TrainOutput(global_step=7176, training_loss=0.061543399250612946, metrics={'train_runtime': 4991.6409, 'train_samples_per_second': 1.438, 'total_flos': 11707190675560800, 'epoch': 12.0})

#Check validation score 

In [ ]:
predictions = trainer.predict(val_dataset)

In [ ]:
preds = predictions.predictions
preds = np.argmax(preds, axis=1)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(Y_val,list(map(int, preds)))
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       247
           1       0.99      0.98      0.98       180
           2       0.97      1.00      0.99        73

    accuracy                           0.99       500
   macro avg       0.98      0.99      0.99       500
weighted avg       0.99      0.99      0.99       500



In [ ]:
df_preds = pd.DataFrame(preds, columns=['BerTweet_Prediction'])
df_id = df_valid['tweet_id']
df_results = pd.concat([df_id, df_preds], join = 'outer', axis = 1)
print(df_results.head())
df_results.to_csv("/content/drive/MyDrive/UPENN/bertweet-results.tsv", sep='\t', index=False)

   tweet_id  BerTweet_Prediction
0     13361                    0
1     20228                    1
2     20120                    2
3     23604                    1
4     31412                    2


In [ ]:
# test
# df_valid = pd.read_csv("/content/drive/MyDrive/UPENN/Task6/valid.tsv", sep='\t')
df_valid = pd.read_csv("/content/drive/MyDrive/UPENN/Task6/test_tweets6.tsv", sep='\t')
df_valid.columns = ["tweet_id", "tweet"]
df_valid["label"] = 0
print(df_valid.head())
# df_valid['label'].replace({"Lit-News_mentions":0, "Nonpersonal_reports":1, "Self_reports":2}, inplace=True)

   tweet_id                                              tweet  label
0         0  @margare44984948 @MattHancock 80 year old rela...      0
1         1  @CurrenScience With the number of cases in the...      0
2         2  @eldivyn I had fever, diarrhoea, loss of sense...      0
3         3  It’s 5 months today since I developed my first...      0
4         4  """""""""""""""For some #COVID19 patients, the...      0


In [ ]:
X_val = np.array(df_valid['tweet'])
Y_val = np.array(df_valid['label'])
X_val = X_val.tolist()
Y_val.tolist()
# Y_val = np.zeros_like(X_val).tolist()
val_encodings = tokenizer(X_val,truncation=True, padding=True)
val_dataset = Task6Dataset(val_encodings, Y_val)


predictions = trainer.predict(val_dataset)


preds = predictions.predictions
preds = np.argmax(preds, axis=1)


df_preds = pd.DataFrame(preds, columns=['covid-twitter-bert-v2_Prediction'])
df_id = df_valid['tweet_id']
df_results = pd.concat([df_id, df_preds], join = 'outer', axis = 1)
print(df_results.head())
df_results.to_csv("/content/drive/MyDrive/UPENN/Task6/bertweet_final.tsv", sep='\t', index=False)

   tweet_id  covid-twitter-bert-v2_Prediction
0         0                                 1
1         1                                 1
2         2                                 2
3         3                                 2
4         4                                 0
